In [ ]:
%pylab inline
%config InlineBackend.figure_format = 'retina'
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
# from ipywidgets import interact
# import ipywidgets as widgets
# import traitlets
import warnings
warnings.filterwarnings("ignore")
# from IPython.display import display, HTML
import os
import subprocess
import time
from cStringIO import StringIO
import pims
from pims import ND2_Reader
#from scipy.ndimage.morphology import grey_dilation
# import PIL
# from PIL import Image
# from PIL.ImageSequence import Iterator
import pandas as pd
from scipy.io import savemat, loadmat
from pandas import json


import tifffile
# import NetTracker
# from NetTracker import TrackingData as TD
# from vidUtils import *
# from batchUtils import getVidFiles

# import MCMClinker
# from MCMClinker import DiffusionFilter as MCMC


from datatank_py.DTDataFile import DTDataFile
from datatank_py.DTProgress import DTProgress
from datatank_py.DTSeries import DTSeriesGroup

matplotlib.style.use('ggplot')
def getFiles(path, kind='tif'):
    kind = '.'+kind
    fn = len(kind)
    filenames = []
    for root, dirs, files in os.walk(path):
        for f in files:
            if f[-fn:] == kind:
                filenames.append(os.path.join(root, f[:-fn]))
    print('Total number of {0} files: '.format(kind[1:]), len(filenames))
    return filenames 
class Make_obj:
    def textureLine(self, d):
        out = '{0} '.format('vt')
        for dn in d:
            out += '{0:1.6g} '.format(dn)
        return out[:-1] + '\n'
    def normalsLine(self, d):
        out = '{0} '.format('vn')
        for dn in d:
            out += '{0} '.format(dn)
        return out[:-1] + '\n'
    def vertexLine(self, d):
        out = 'v '
        for dn in d:
            out += '{0} '.format(dn)
        return out[:-1] + '\n'
    def faceLine(self, d):
        out = 'f '
        for dn in d:
            out += '{0}/{0}/{0} '.format(dn)
        return out[:-1] + '\n'
    def __call__(self, points, faces=[], normals=[], values=[], objectLabel=1):
        #out = 'o {0}\n'.format(objectLabel)
        out = ''
        assert array(faces).size == 0 or array(faces).shape[1] == 3
        assert array(points).shape[1] == 3
        if 0 in faces:
            faces += 1
        for p in points:
            out += self.vertexLine(p)
        for n in normals:
            out += self.normalsLine(n)
        for v in values:
            out += self.textureLine((v, 0, 0))
        for face in faces:
            out += self.faceLine(face)
        return out
path = '/Users/jaynewby/Dropbox/Gladfelter/'
# savedir = 'GEMS-Whi3dpQ-june7'
# savedir = 'GEMS-june11'
savedir = 'GEMS-whi3tom'
# savedir = 'GEMS-nuclei-june18'
# savedir = 'GEMS-wbs'
savePath = os.path.join(path, savedir)

# Save GEMS json data into directory of CSV files, folders with DataTank placeholders, and video metadata

In [ ]:
# CloudBasePath = 'gs://pipeline-proc'
# tset = 'Wed Jun  6 20_26_16 2018--nd2.json'
# tset = 'Wed Jun  6 21_50_38 2018--nd2.json'
# tset = 'Thu Jun  7 14_00_59 2018--nd2.txt'
tset = 'trackingData.txt'
jsonfn = os.path.join(savePath, tset)
with open(jsonfn, 'r') as jfile:
    for line in jfile:
        data = json.loads(line)
        fn = data.keys()[0]
        vdata = data[fn]
        _, nameWithExt = os.path.split(fn)
        name, ext = os.path.splitext(nameWithExt)
        print(os.path.join(savePath, name))

        if not os.path.isdir(savePath):
            os.makedirs(savePath)

        pdata = vdata['particleSet']
        x, y, z = pdata['x'], pdata['y'], pdata['z']
        t, p, r = pdata['t'], pdata['p'], pdata['r']
        data = array([x, y, z, t, p, r])
        df = pd.DataFrame(data.T, columns=['x', 'y', 'z', 't', 'p', 'r'])\
                .astype({'t': 'uint16'})

        df.to_csv(os.path.join(savePath, name + ' (localizations).csv'))

        ldata = vdata['tracks']
        Uparticles = sort(array([int(v) for v in ldata.keys()], 'int'))
        t, x, y, z, p, r = [], [], [], [], [], []
        particle = []
        for pn in Uparticles:
            f = ldata[str(pn)]
            x.extend(f['x'])
            y.extend(f['y'])
            z.extend(f['z'])
            p.extend(f['p'])
            r.extend(f['r'])
            t.extend(f['t'])
            particle.extend(pn*ones(len(f['x']), 'int'))
        data = array([particle, t, x, y, z, p, r])
        df = pd.DataFrame(
                data.T, 
                columns=['particle', 'frame', 'x', 'y', 'z', 'p', 'r'])\
                .astype({'particle': 'int', 'frame': 'int'})\
                .sort_values(['particle', 'frame'], axis=0)
        df.to_csv(os.path.join(savePath, name + ' (tracks).csv'))
        #########################
        md = vdata['metadata']
        #c = md['conversions']
        md['dt'] = 1.0
        NF, Ny, Nx, Nz = md['vidShape']
        print(NF, Ny, Nx, Nz)
        outdict = {'Seq_NF': array([u'Real Number'], dtype='<U11'),
                   'Seq_Nx': array([u'Real Number'], dtype='<U11'),
                   'Seq_Ny': array([u'Real Number'], dtype='<U11'),
                   'Seq_Nz': array([u'Real Number'], dtype='<U11'),
                   'Seq_dxy': array([u'Real Number'], dtype='<U11'),
                   'Seq_dz': array([u'Real Number'], dtype='<U11'),
                   'Seq_dt': array([u'Real Number'], dtype='<U11'),
                   'NF': array([[NF]], dtype='int32'),
                   'Nx': array([[Nx]], dtype='int32'),
                   'Ny': array([[Ny]], dtype='int32'),
                   'Nz': array([[Nz]], dtype='int32'),
                   'dxy': array([[md['dxy']]], dtype='float32'),
                   'dz': array([[md['dz']]], dtype='float32'),
                   'dt': array([[md['dt']]], dtype='float32')}
        #savemat(os.path.join(savePath, name + '-videoMetaData.mat'), outdict, format='4')
        placeHolderPath = os.path.join(savePath, name)
        if not os.path.isdir(placeHolderPath):
            os.mkdir(placeHolderPath)
        for t in arange(NF):
            with open(os.path.join(placeHolderPath, '{0}.dtph'.format(t)), 'w') as file:
                file.write('nothing')

# Get metadata from the Gladfelter lab drive

In [ ]:
import pims
GladPath = '/Volumes/GladfelterLab/Erin/GEMS/jay/'
folder = 'WBS GEMS'
assert len(folder) > 0
GladPath += folder
if not os.path.isdir(savePath):
    os.makedirs(savePath)
files = getFiles(GladPath, kind='nd2')
for fn in files:
    _, name = os.path.split(fn)
    with pims.open(fn+'.nd2') as frames:
        outDict = {}
        print(name)
        mdVid = frames.metadata
        Nx = frames.sizes['x']
        Ny = frames.sizes['y']
        Nz = frames.sizes['z']
        try:
            NF = frames.sizes['t']
        except:
            continue
        print(NF, Ny, Nx, Nz)
        md0 = frames[0].metadata
        md1 = frames[1].metadata
        dt = mean(array(md1['t_ms']) - array(md0['t_ms']))/1000.
        dxy = frames.calibration
        dz = frames.calibrationZ
        zscale = frames.calibrationZ/frames.calibration
        outdict = {
            'Seq_NF': array([u'Real Number'], dtype='<U11'),
            'Seq_Nx': array([u'Real Number'], dtype='<U11'),
            'Seq_Ny': array([u'Real Number'], dtype='<U11'),
            'Seq_Nz': array([u'Real Number'], dtype='<U11'),
            'Seq_dxy': array([u'Real Number'], dtype='<U11'),
            'Seq_dz': array([u'Real Number'], dtype='<U11'),
            'Seq_dt': array([u'Real Number'], dtype='<U11'),
            'NF': array([[NF]], dtype='int32'),
            'Nx': array([[Nx]], dtype='int32'),
            'Ny': array([[Ny]], dtype='int32'),
            'Nz': array([[Nz]], dtype='int32'),
            'dxy': array([[dxy]], dtype='float32'),
            'dz': array([[dz]], dtype='float32'),
            'dt': array([[dt]], dtype='float32')}
        savemat(
            os.path.join(
                savePath, 
                name + '-videoMetaData.mat'), 
            outdict, 
            format='4')

# Save points in obj file for hand segmentation

In [ ]:
objSaveDir = 'HyphaDensityProject/{0} (points for segmentation)'.format(savedir)
files = [fn for fn in getFiles(savePath, kind='csv') if '(tracks)' in fn]
if not os.path.isdir(objSaveDir):
    os.mkdir(objSaveDir)
for fn in files:
    _, name = os.path.split(fn)
    data = pd.read_csv(fn + '.csv')
    points = array(data[['x', 'y', 'z']])
    with open(os.path.join(objSaveDir, name + '.obj'), 'w+b') as ofile:
        print(ofile.name)
        ofile.write(Make_obj()(points))

# Convert segmented txt files to csv

In [ ]:
ObjPath = '/Users/jaynewby/Dropbox/Gladfelter/HyphaDensityProject/'
ObjPath += savedir + ' (points for segmentation)/segmented/'
files = [fn for fn in getFiles(ObjPath, kind='txt') 
             if 'CSVpointsViewer' not in fn]
vnames = []
for fn in files:
    bp1, name = os.path.split(fn)
    bp2, vname = os.path.split(bp1)
    vnames.append(vname)
    pointData = []
    with open(fn + '.txt', 'r') as ofile:
        lines = ofile.read().splitlines()
    for l in lines:
        s = l.split()
        if len(s) > 0:
            pointData.append(float64(s[:3]))
    df = pd.DataFrame(array(pointData), columns=['x', 'y', 'z'])
    df.to_csv(fn + '.csv', index=False)
vnames = set(vnames)
for vname in vnames:
    combinedData = []
    for fn in files:
        base, name = os.path.split(fn)
        _, dirname = os.path.split(base)
        if not vname == dirname:
            continue
        if not 'GEMS 1_17' in ObjPath:
            segmentNumber = int(name)
        else:
            segmentNumber = int(name.split('_')[1])
        with open(fn + '.txt', 'r') as ofile:
            lines = ofile.read().splitlines()
        for l in lines:
            s = l.split()
            if len(s) > 0:
                pdata = r_[float64(s[:3]), segmentNumber]
                combinedData.append(pdata)
    df = pd.DataFrame(array(combinedData), columns=['x', 'y', 'z', 's'])
    if savedir == 'GEMS-wbs':
        vname = '0' + vname[1:]
    df.to_csv(
        os.path.join(
            ObjPath, 
            vname + ' (point segments).csv'), 
        index=False)
    df.to_csv(
        os.path.join(
            savePath, 
            vname + ' (point segments).csv'), 
        index=False)

# Output surfaces as OBJ for manual touchups

In [ ]:
ObjPath = '/Users/jaynewby/Dropbox/Gladfelter/HyphaDensityProject/'
ObjPath += savedir + ' (points for segmentation)/surface meshes for touchup/'
ObjPath2 = '/Users/jaynewby/Dropbox/Gladfelter/HyphaDensityProject/'
ObjPath2 += savedir + ' (points for segmentation)/surface meshes refinished/'
files = [fn for fn in getFiles(savePath, kind='dtbin') if 'SurfacesOnly' in fn]
if not os.path.isdir(ObjPath):
    os.mkdir(ObjPath)
if not os.path.isdir(ObjPath2):
    os.mkdir(ObjPath2)
for fn in files:
    _, nameRaw = os.path.split(fn)
    name = nameRaw.split('-hyphaeList-SurfacesOnly')[0]
    opath = os.path.join(ObjPath, name)
    opath2 = os.path.join(ObjPath2, name)
    print('---------')
    print(opath)
    if not os.path.isdir(opath):
        os.mkdir(opath)
    if not os.path.isdir(opath2):
        os.mkdir(opath2)
    with DTDataFile(fn + '.dtbin', readonly=True) as sfile:
        #print(sfile.variable_names())
        #print([k for k in sfile.variable_names() if 'surface_P' in k])
        Nsurfs = len([k for k in sfile.variable_names() 
                          if 'surface_P' in k and not 'bbox' in k])
        for n in arange(Nsurfs):
            points = sfile['Var_{0}_surface_P'.format(n)][0]
            triangles = sfile['Var_{0}_surface'.format(n)][0]
            with open(os.path.join(opath, '{0}.obj'.format(n+1)), 'w+b') as ofile:
                ofile.write(Make_obj()(points, faces=triangles))
            ## save a coppy in the refinished folder if there is no file in there already
            ropath = os.path.join(opath2, '{0}.obj'.format(n+1))
            if not os.path.exists(ropath):
                with open(ropath, 'w+b') as ofile:
                    ofile.write(Make_obj()(points, faces=triangles))

# Import new surface meshes into DataTank

In [ ]:
ObjPath = '/Users/jaynewby/Dropbox/Gladfelter/HyphaDensityProject/'
ObjPath += savedir + ' (points for segmentation)/surface meshes refinished/'
files = [fn for fn in getFiles(savePath, kind='dtbin') if 'SurfacesOnly' in fn]
for fn in files:
    bp, nameRaw = os.path.split(fn)
    name = nameRaw.split('-hyphaeList-SurfacesOnly')[0]
    opath = os.path.join(ObjPath, name)
    with DTDataFile(fn + '.dtbin', readonly=True) as sfile:
        Nsurfs = len([k for k in sfile.variable_names() 
                          if 'surface_P' in k and not 'bbox' in k])
    with DTDataFile(
            os.path.join(
                bp, 
                name + '-newSurfaces.dtbin'), 
            truncate=True) as sfile:
        S = DTSeriesGroup(sfile, 'Var', {'points': 'Array', 'triangles': 'Array'})
        for n in arange(Nsurfs):
            points, triangles = [], []
            with open(os.path.join(opath, '{0}.obj'.format(n+1)), 'r') as ofile:
                lines = ofile.read().splitlines()
            for l in lines:
                s = l.split()
                if len(s) > 0 and s[0] == 'v':
                    points.append(float64(s[1:4]))
                if len(s) > 0 and s[0] == 'f':
                    tris = [tri.split('/')[0] for tri in s[1:]]
                    triangles.append(int32(tris) - 1)
            S.add(
                float(n), 
                {'points': array(points), 
                 'triangles': array(triangles)})


# Load the nuclei tracking json file

In [ ]:
# Njsonfn = '/Users/jaynewby/Dropbox/Gladfelter/GEMS 1_17/Tue Mar 20 21_38_56 2018--nd2.json'
# NjsonData = {}
# with open(Njsonfn, 'r') as jfile:
#     raw = jfile.read().splitlines()
# for l in raw:
#     NjsonData.update(json.loads(l))
# for key in NjsonData:
#     print(key)

# Create 3D tetrahedral mesh

In [ ]:
# import pymesh
# # input_mesh = pymesh.generate_icosphere(1.0, [0.0, 0.0, 0.0])
# filename = '/Users/jaynewby/Dropbox/Gladfelter/HyphaDensityProject/GEMS 1_17 (points for segmentation)/surface meshes refinished/1016/3.obj'
# # filename = 'testMesh2.obj'
# input_mesh = pymesh.load_mesh(filename)
# tetgen = pymesh.tetgen()
# tetgen.points = input_mesh.vertices # Input points.
# tetgen.triangles = input_mesh.faces # Input triangles
# tetgen.max_tet_volume = inf
# tetgen.verbosity = 2
# tetgen.run() # Execute tetgen
# mesh = tetgen.mesh # Extract output tetrahedral mesh.

# Retrack nuclei, filtering for size and using MCMC linking

In [ ]:
# reload(MCMC)
def DiffusionLink(trackData, NmaxElements, coastLimit=1, CAcost=7.5, Nsamples=1, reverse=False):
    Pnew = 0.5#*ones(trackData.shape, 'float16')
    F = MCMC.DiffusionFilter(Nsamples, NmaxElements, trackData, Pnew, 
                               pars, zscale,
                               cutOff=40., coastLimit=coastLimit, CAcost=CAcost)
    print('linking')
    Data = F.track(reverse)
    trackData = TD.TrackingData(shape=vidShape).setData(Data)
    trackData._trajectoryStats()
    oneF = sum([g.frame.size == 1 for p, g in trackData.Data.groupby('particle')])
    print('number of 1-frame tracks:', oneF)
    print('')
    return trackData
jsonfn = os.path.join(savePath, 'nuclei.json')
with open(jsonfn, 'r') as jfile:
    for line in jfile:
        data = json.loads(line)
        key = data.keys()[0]
        vdata = data[key]

        _, nameWithExt = os.path.split(key)
        name, ext = os.path.splitext(nameWithExt)
        print('--------------------')
        print(name)
        print('--------------------')
        fn = os.path.join(savePath, name)
        vidMD = loadmat(fn + '-videoMetaData.mat')
        vidShape = (int(vidMD['NF'].squeeze()), 
                    int(vidMD['Ny'].squeeze()/2), 
                    int(vidMD['Nx'].squeeze()/2), 
                    int(vidMD['Nz'].squeeze()))
        if vidShape[0] <= 50:
            continue
        zscale = (vidMD['dz']/(2.*vidMD['dxy'])).squeeze()
        pdata = vdata['particleSet']
        x, y, z, t, p, r = pdata['x'], pdata['y'], pdata['z'], pdata['t'], pdata['p'], pdata['r']
        data = array([x, y, z, t, p, r])[:, array(r) > 4.]
        df = pd.DataFrame(data.T, columns=['x', 'y', 'z', 't', 'p', 'r'])\
                .astype({'t': 'uint16'})
        print('shape:', vidShape)
        trackData = TD.TrackingData(shape=vidShape, zscale=zscale)
        trackData.particleSet = df
        trackData.particleSetGrouped = trackData._makePSgrouped()
        trackData.linkParticles(D=0.2)
        # trackData.DTsave(fn + '-forDT.mat')
        trackData.Data.to_csv(fn + ' (nuclei).csv')

        D = 0.05
        pars = (D,)

        NmaxElements = int(3*trackData.Nparticles)
        print('----')
        C = 10.

    #     trackData.filterPathsByLength(5)
        CL = [5, 5, 5, 7, 10, 20]
        for n in arange(6):
            trackDataHS = DiffusionLink(trackData, NmaxElements, coastLimit=CL[n], CAcost=C, Nsamples=2)
            trackDataHS.filterPathsByLength(5)
            NmaxElements = int(4*trackDataHS.Nparticles)
            C *= 1.2
        trackDataHS.filterPathsByLength(50)
        trackDataHS._trajectoryStats()
        trackDataHS.Data.to_csv(fn + ' (MCMC)(nuclei).csv')

# Move axial density data to Grace's working folder

In [ ]:
files = [fn for fn in getFiles(savePath, kind='dtbin') if 'hyphaAxisDensity' in fn]
ObjPath = '/Users/jaynewby/Dropbox/Gladfelter/HyphaDensityProject/'
ObjPath += savedir + ' (axialDensityData)/x10t10'
if not os.path.isdir(ObjPath):
    os.makedirs(ObjPath)
for fn in files:
    _, name = os.path.split(fn)
    outFile = os.path.join(ObjPath, name + '.dtbin')
    with open(fn + '.dtbin', 'rb') as afile, open(outFile, 'w+b') as sfile:
        sfile.write(afile.read())
        
# glob = os.path.join(savePath, '*hyphaAxisDensity.dtbin')
# !cp {glob} {ObjPath}

# Create TIF max projection files from Beam output files

In [ ]:
files = [fn for fn in getFiles(savePath, kind='txt') if 'maxProjectionData' in fn]
def procFn(pfile):
    data = ''
    name = pfile.readline()
    assert name[:5] == 'gs://'
    while True:
        d = pfile.readline()
        if len(d) == 0:
            yield name, data
            break
        elif d[:5] == 'gs://':
            yield name, data
            name = d
            data = ''
        else:
            data += d
    
for fn in files[1:]:
    data = ''
    with open(fn+'.txt', 'rb') as pfile:
        for nm, data in procFn(pfile):
            _, name = os.path.split(nm[:-1])
            with open(os.path.join(savePath, name+'.tif'), 'w+b') as sfile:
                sfile.write(data)

# Misc Stuff

## Extract pixel size and framerate

In [ ]:
files = [fn for fn in getFiles(savePath, kind='mat') if 'videoMetaData' in fn]
DT = {}
DXY = {}
for fn in files:
    md = loadmat(fn+'.mat')
    _, name = os.path.split(fn)
    name = name.replace('-videoMetaData', '')
    DT[name] = 1.*md['dt'].squeeze()
    DXY[name] = 1.*md['dxy'].squeeze()
print(DT)
print('------')
print(DXY)

## Total size of video files

In [ ]:
import os
from apache_beam.io.filesystems import FileSystems
match_result = FileSystems.match(
    [os.path.join('gs://gems-drive/GEMS/', '**.nd2')])[0]
files_metadata = match_result.metadata_list
total_size = 0
for md in files_metadata:
    total_size += md.size_in_bytes
print(total_size/1e12)

## colorbar figure

In [ ]:
cmapfile = '/Users/jaynewby/Dropbox/DataTankCode/more_cmaps/rainbow_bgyr_35-85_c73_n256.csv'
cdf = pd.read_csv(cmapfile, names=['x', 'y', 'z'])
cm0 = matplotlib.pylab.get_cmap('jet')
cm = cm0.from_list('rbow', float64(array(cdf))/255.)
matplotlib.style.use('dark_background')
scale = array([1e-4, 1e-3, 1e-2, 1e-1])
labels = [r'$10^{-4}$',r'$10^{-3}$',r'$10^{-2}$',r'$10^{-1}$']
X, Y = meshgrid(linspace(0, 1.05), linspace(0, 1.05))
figure(1, [1, 10])
pcolor(X, Y, Y, cmap=cm)
xticks([])
yticks([0, 0.33, 0.66, 1], labels, fontsize=24)
title(r'$\mu \rm{m}^2/\rm{sec}$', fontsize=24)